##  This is a script to use deep learning to predict stuff

# Do Imports

In [72]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import confusion_matrix
import math

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time

import matplotlib.pyplot as plt

# Read data and create Dataset iterators

In [84]:
# Read in data (from pkl file probably)

#os.getcwd()
df = pd.read_pickle('../data/price_level_total_view_2017-01-03_AAPL_grouped_1')
df.head()
df.tail()

,trade_volume,mid_price_log,target,trade_volume_differential,mid_price_log_differential_0_1,mid_price_log_differential_0_2,mid_price_log_differential_0_3,mid_price_log_differential_1_2,mid_price_log_differential_1_3,mid_price_log_differential_2_3,trade_volume_differential_0_1,trade_volume_differential_0_2,trade_volume_differential_0_3,trade_volume_differential_1_2,trade_volume_differential_1_3,trade_volume_differential_2_3
groupid,,,,,,,,,,,,,,,,
57599089,4659293,25.491830,2,0.0,2,2,2,0,0,1,1,1,1,1,1,1
57599090,4659293,25.550179,0,0.0,2,2,2,2,2,0,1,1,1,1,1,1
57599091,4659293,25.372644,2,0.0,0,0,0,2,2,2,1,1,1,1,1,1
57599092,4659293,25.571578,0,0.0,2,2,2,0,0,2,1,1,1,1,1,1
57599093,4659293,24.424568,0,0.0,0,0,0,2,2,0,1,1,1,1,1,1


In [97]:
dfX = df.iloc[1:,[1,3,4,5,6,10,11,12]]
dfy = df.iloc[1:,2]

In [98]:
n=dfy.shape[0]
cutoff = np.floor(n * 0.8).astype(int)

In [101]:
#_1, _2 = (dfy.iloc[0:cutoff , :].values.ravel() , dfy.iloc[cutoff: , :].values.ravel() )
#_1, _2 = (dfy.iloc[0:cutoff].values.ravel() , dfy.iloc[cutoff:].values.ravel() )

In [ ]:
# temporary object to develop to
trainarray = np.zeros([10,5,2], dtype=float) #[n_rows, window_size, features]
labelarray = np.zeros([10,2], dtype=float) # [n_rows, output_classes]

testarray = np.zeros([10,5,2], dtype=float) #[n_rows, window_size, features]
testlabelarray = np.zeros([10,2], dtype=float) # [n_rows, output_classes]


# Define functions

In [102]:
def features_2d_to_3d(data, labels, window):
    data_n, data_w = data.shape
    stride1, stride2 = data.strides
    new_len = data_n - window
    data3d = as_strided(data, [new_len , window, data_w], strides=[stride1, stride1, stride2])
    return(data3d, labels[:len(labels)-window])

def flatten_3d(data):
    data_n = data.shape[0]
    new_width = data.shape[1]*data.shape[2]
    
    return np.reshape(data, (data_n, new_width))
    
def split_data(dfX, dfy, train_frac):
    
    #X = df.iloc[:,0:2]
    #y = df.iloc[:,2:3]
    X = dfX
    y = dfy
    n = X.shape[0]
    cutoff = np.floor(n * train_frac).astype(int) # total - the number you want to test, which here i'm flooring 
    #                   (amount you want in training should be 1/10th value the denominator)
    # cutoff

    X_train, X_test = (X.iloc[0:cutoff , :] , X.iloc[cutoff: , :] )

    y_train, y_test = (y.iloc[0:cutoff].values.ravel() , y.iloc[cutoff:].values.ravel() )
    
    return X_train, y_train, X_test, y_test


In [11]:
def create_datasets(trainarray, labelarray, testarray, testlabelarray, batch_size):
    tf.reset_default_graph()
    
    train_n = trainarray.shape[0]
    test_n = testarray.shape[0]

    with tf.name_scope("dataset"):
        training_dataset = (
            tf.data.Dataset.from_tensor_slices(
                (
                    tf.cast(trainarray, tf.float32),
                    tf.cast(labelarray, tf.int32)
                )
            ).shuffle(buffer_size=2*train_n).batch(batch_size) # multiply by 2 if using accuracy calc
        )

        test_dataset = (
            tf.data.Dataset.from_tensor_slices(
                (
                    tf.cast(testarray, tf.float32),
                    tf.cast(testlabelarray, tf.int32)
                )
            )
        ).shuffle(buffer_size=2*test_n).batch(batch_size)

    with tf.name_scope("iterator"):
        iterator = tf.data.Iterator.from_structure(training_dataset.output_types, training_dataset.output_shapes)
        features, labels = iterator.get_next()
        train_init = iterator.make_initializer(training_dataset) # initializer for train_data
        test_init = iterator.make_initializer(test_dataset) # initializer for train_data

    return features, labels, train_init, test_init, trainarray.shape[1]

# Define model

In [ ]:
def create_model(features, labels, n_inputs, n_outputs=3):
    keep_prob = tf.placeholder(tf.float32)
    
    with tf.name_scope("dnn"):
    
        hidden1 = tf.layers.dense(features, n_inputs, name="hidden1",
                                  activation=tf.nn.elu)#, kernel_regularizer=regularizer1)
        hidden2 = tf.layers.dense(hidden1, n_inputs, name="hidden2",
                                  activation=tf.nn.elu)#, kernel_regularizer=regularizer2)

        #max1 = tf.contrib.layers.maxout(hidden1, n_inputs//2)

    #     dropout1 = tf.nn.dropout(hidden1, keep_prob)
    #     hidden2 = tf.layers.dense(dropout1, n_hidden2, name="hidden2",
    #                               activation=tf.nn.relu)#, kernel_regularizer=regularizer2)
    #     dropout2 = tf.nn.dropout(hidden2, keep_prob)
    #     hidden3 = tf.layers.dense(dropout2, n_hidden3, name="hidden3",
    #                               activation=tf.nn.relu)#, kernel_regularizer=regularizer3)
    #     dropout3 = tf.nn.dropout(hidden3, keep_prob)
    #     hidden4 = tf.layers.dense(dropout3, n_hidden4, name="hidden4",
    #                               activation=tf.nn.relu)#, kernel_regularizer=regularizer4)
    #     dropout4 = tf.nn.dropout(hidden4, keep_prob)
    #     hidden5 = tf.layers.dense(dropout4, n_hidden5, name="hidden5",
    #                               activation=tf.nn.relu)#, kernel_regularizer=regularizer4)
    #     dropout5 = tf.nn.dropout(hidden5, keep_prob)
    #     hidden6 = tf.layers.dense(dropout5, n_hidden5, name="hidden6",
    #                               activation=tf.nn.relu)#, kernel_regularizer=regularizer4)
    #     dropout6 = tf.nn.dropout(hidden6, keep_prob)
    #     hidden7 = tf.layers.dense(dropout6, n_hidden5, name="hidden7",
    #                               activation=tf.nn.relu)#, kernel_regularizer=regularizer4)
    #     dropout7 = tf.nn.dropout(hidden7, keep_prob)
        logits = tf.layers.dense(hidden2, n_outputs, name="outputs")#, kernel_regularizer=regularizer5)
    

    with tf.name_scope("loss"):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")
    #     l2_loss = tf.reduce_sum(tf.losses.get_regularization_losses())
    #     loss += l2_loss
        loss_summary = tf.summary.scalar('log_loss', loss)

    learning_rate = 0.002

    with tf.name_scope("train"):
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)

    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, labels, 1)
        accuracy = tf.reduce_sum(tf.cast(correct, tf.float32))
        accuracy_summary = tf.summary.scalar('accuracy', accuracy)

    with tf.name_scope("predict"):
        output = tf.nn.softmax(logits)

    merged = tf.summary.merge_all()
    
    return training_op, output, loss, accuracy




In [130]:
def create_cnn_model(features, labels, n_inputs, n_outputs=3):
    keep_prob = tf.placeholder(tf.float32)
    
    with tf.name_scope("cnn"):
        
        input_layer = tf.reshape(features, [-1, n_inputs[0], n_inputs[1], 1])
                       
        # Convolutional Layer #1
        conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=32,
            kernel_size=[5, n_inputs[1]],
            strides=[1,1],
            padding="same",
            activation=tf.nn.relu)
                       
        conv1_flat = tf.reshape(conv1, [-1, n_inputs[0] * n_inputs[1] * 32])
    
        hidden1 = tf.layers.dense(conv1_flat, n_inputs[0], name="hidden1",
                                  activation=tf.nn.elu)#, kernel_regularizer=regularizer1)
        hidden2 = tf.layers.dense(hidden1, n_inputs[0], name="hidden2",
                                  activation=tf.nn.elu)#, kernel_regularizer=regularizer2)
        logits = tf.layers.dense(hidden2, n_outputs, name="outputs")#, kernel_regularizer=regularizer5)
    

    with tf.name_scope("loss"):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")
    #     l2_loss = tf.reduce_sum(tf.losses.get_regularization_losses())
    #     loss += l2_loss
        loss_summary = tf.summary.scalar('log_loss', loss)

    learning_rate = 0.002

    with tf.name_scope("train"):
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)

    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, labels, 1)
        accuracy = tf.reduce_sum(tf.cast(correct, tf.float32))
        accuracy_summary = tf.summary.scalar('accuracy', accuracy)

    with tf.name_scope("predict"):
        output = tf.nn.softmax(logits)

    merged = tf.summary.merge_all()
    
    return training_op, output, loss, accuracy


In [61]:
def get_predictions(te_init, test_n, output_op):
    with tf.Session() as sess:
        preds = np.zeros(test_n)
        sess.run(tf.global_variables_initializer())
        sess.run(te_init) # drawing samples from test_data
        try:
            for i in range(test_n):
            #for i in range(testarray.shape[0]):
                pred = sess.run(output_op) #, feed_dict={keep_prob : 1}) # for dropout only
                preds[i] = np.argmax(pred)
        except tf.errors.OutOfRangeError:
            pass
        print(preds)
        
        return preds

# Train model

In [103]:
# Create train/test sets
trainarray,labelarray,testarray,testlabelarray = split_data(dfX, dfy, 0.8)

In [104]:
# transform data set to use data windows
from numpy.lib.stride_tricks import as_strided

trainarray, labelarray = features_2d_to_3d(np.array(trainarray), np.array(labelarray), 5)
testarray, testlabelarray = features_2d_to_3d(np.array(testarray), np.array(testlabelarray), 5)
# trainarray = flatten_3d(trainarray) # don't need to flatten for CNN
# testarray = flatten_3d(testarray)

TensorShape([Dimension(None), Dimension(5), Dimension(8)])

In [105]:
# Do training
batch_size = 100
train_n = trainarray.shape[0]
test_n = testarray.shape[0]
n_batches = train_n // batch_size
n_batches_test = test_n // batch_size

In [131]:
# Create datasets & model



features, labels, tr_init, te_init, n_inputs = create_datasets(trainarray, labelarray, testarray, testlabelarray, batch_size)
#training_op, output, loss, accuracy = create_model(features, labels, n_inputs, n_outputs=3)
n_inputs = [trainarray.shape[1], trainarray.shape[2]]
training_op, output, loss, accuracy = create_cnn_model(features, labels, n_inputs, n_outputs=3)



In [129]:
n_inputs[0], n_inputs[1]

(5, 8)

[5, 2]

In [138]:
n_epochs = 1

columns = ['t-plus', 'loss', 'accuracy', 'test_loss', 'test_accuracy']
summaries = pd.DataFrame(np.zeros([n_epochs,5], dtype=float), columns=columns)
run_name = 'model1'



with tf.Session() as sess:
    start_time = time.time()
    writer = tf.summary.FileWriter('./', tf.get_default_graph())
    writer.add_graph(sess.graph)
    
    sess.run(tf.global_variables_initializer())

    tot_batches_run = 0
    for epoch in range(n_epochs):
        sess.run(tr_init) # drawing samples from train_data
        tot_loss = 0
        for i in range(n_batches):
            try:
                _, loss_value = sess.run([training_op, loss]) # , feed_dict={keep_prob : 0.75} # for dropout only
                tot_loss += loss_value
            except tf.errors.OutOfRangeError:
                print("out of range on iter {}".format(i))
                break
        
        
        # Now gauge training accuracy
        sess.run(tr_init) # drawing samples from test_data
        total_correct_preds = 0
        try:
            while True:
                accuracy_batch = sess.run(accuracy) # , feed_dict={keep_prob : 1} # for dropout only
                total_correct_preds += accuracy_batch
        except tf.errors.OutOfRangeError:
            pass
        tr_acc = total_correct_preds/train_n
        
        # Now get testing loss
        sess.run(te_init) # drawing samples from test_data
        test_tot_loss = 0
        for i in range(n_batches_test):
            try:
                loss_value = sess.run([loss]) # , feed_dict={keep_prob : 0.75} # for dropout only
                test_tot_loss += loss_value[0]
            except tf.errors.OutOfRangeError:
                print("out of range on iter {}".format(i))
                break
        
        # Now gauge testing accuracy
        sess.run(te_init) # drawing samples from test_data
        total_correct_preds = 0
        try:
            while True:
                accuracy_batch = sess.run(accuracy) # , feed_dict={keep_prob : 1} # for dropout only
                total_correct_preds += accuracy_batch
        except tf.errors.OutOfRangeError:
            pass
        te_acc = total_correct_preds/test_n

        
        epoch_time = time.time()
        print("Epoch: {}, Train_Loss: {:.4f}, Test_Loss: {:.4f}, Train_Accuracy: {:.4f}, Test_Accuracy: {:.4f}"\
              .format(epoch, tot_loss, test_tot_loss, tr_acc, te_acc))
        cum_time = epoch_time - start_time
        summaries.iloc[epoch,:] = cum_time, tot_loss, test_tot_loss, tr_acc, te_acc

        writer.close()
     

Epoch: 0, Train_Loss: 271.0470, Test_Loss: 19.7518, Train_Accuracy: 0.9951, Test_Accuracy: 0.9856


In [114]:
pd.Series(p).unique()

array([138.,  78.,  75., 264., 150., 216.,  12.,  69.,  87., 228.,  57.,
       249.,  93.,   9.,  81., 156., 252.,  39., 111., 132., 144., 123.,
       294., 189.,  27., 243., 183., 222., 147., 174., 261.,  66., 291.,
        63., 276., 198.,  42.,   6.,  51., 207., 255.,  21.,  99.,  96.,
       195.,  30., 105., 177.,  60., 135., 204., 210., 297.,  48., 114.,
        18., 282.,  33.,  36.,   0., 225., 159., 171., 120.,  15.,  45.,
       180.,   3.,  24., 126.,  90., 240., 129., 117., 192., 141., 165.,
       168.,  72., 231.,  54., 285., 201., 246.,  84., 153., 279., 288.,
       237., 267., 108., 258., 234., 162., 219., 273., 102., 270., 186.,
       213.])

In [109]:
# Make predictions
p = get_predictions(te_init, test_n, output)

# actual along left, predicted along top
confusion_matrix(testlabelarray, p)





[138.  78.  75. ...   0.   0.   0.]


array([[13226,     0,     0, ...,     1,     2,     1],
       [32670,     0,     0, ...,     6,     3,     1],
       [13220,     0,     0, ...,     0,     1,     2],
       ...,
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0]])

# Closing remarks